In [17]:
# 📦 Install dependencies
!pip install requests pandas

In [52]:
import pandas as pd
import requests
import time
import json
from datetime import datetime

In [54]:
# 🔐 Insert your API key here
API_KEY = "e8b43912-6603-413c-b544-3ca7f47cd06b"

# 🧠 Function to format headers
def get_headers(api_key):
    return {
        "Authorization": f"Bearer {api_key}",
        "Accept": "application/json"
    }


In [55]:
def list_sports_markets(api_key):
    url = "https://api.elections.kalshi.com/trade-api/v2/markets/"
    headers = get_headers(api_key)
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print("❌ Error fetching market list:", response.status_code, response.text)
        return []

    markets = response.json().get("markets", [])
    simplified = [(m["ticker"], m["title"]) for m in markets]
    
    print(f"✅ Found {len(simplified)} markets:")
    for ticker, title in simplified:
        print(f"{ticker:40} → {title}")
    
    return simplified

In [44]:
list_election_markets('e8b43912-6603-413c-b544-3ca7f47cd06b')

🔍 Status: 200
🔍 URL: https://api.elections.kalshi.com/trade-api/v2/markets?category=sports
🔍 Full Response:
{
  "cursor": "CgsI27PMxwYQoNetTBI2S1hNVkVORkxNVUxUSUdBTUVFWFRFTkRFRC1TMjAyNTNFRDZFQTc1NDMyLTcxODI0RkE4MTM3",
  "markets": [
    {
      "can_close_early": true,
      "category": "",
      "close_time": "2025-11-01T16:00:00Z",
      "custom_strike": {
        "tennis_competitor": "f0c27486-2bf2-4d48-b064-66acd05967db"
      },
      "early_close_condition": "This market will close and expire after a winner is declared.",
      "event_ticker": "KXATPMATCH-25OCT18SHARUU",
      "expected_expiration_time": "2025-10-18T19:00:00Z",
      "expiration_time": "2025-11-01T16:00:00Z",
      "expiration_value": "",
      "last_price": 0,
      "last_price_dollars": "0.0000",
      "latest_expiration_time": "2025-11-01T16:00:00Z",
      "liquidity": 738849,
      "liquidity_dollars": "7388.4900",
      "market_type": "binary",
      "no_ask": 75,
      "no_ask_dollars": "0.7500",
      "no_

In [45]:
# 🎯 Define your market and date range
MARKET_TICKER = "KXATPMATCH-25OCT18SHARUU"  # replace as needed
START_DATE = "2025-09-01T00:00:00Z"
END_DATE   = "2025-11-01T16:00:00Z"


In [56]:
def fetch_current_orderbook(ticker, api_key):
    url = f"https://api.elections.kalshi.com/trade-api/v2/markets/{ticker}/orderbook/"
    headers = get_headers(api_key)
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print("❌ API Error:", response.status_code, response.text)
        return None

    return response.json()


In [57]:
# Run this to list available markets
markets = list_sports_markets(API_KEY)

# Paste in your selected ticker from the list above
MARKET_TICKER = "PASTE_A_VALID_TICKER_HERE"

# Fetch live order book data
ob_data = fetch_current_orderbook(MARKET_TICKER, API_KEY)

# Display it
print(json.dumps(ob_data, indent=2))

✅ Found 100 markets:
KXATPMATCH-25OCT18SHARUU-SHA             → Will Denis Shapovalov win the Shapovalov vs Ruud match?
KXATPMATCH-25OCT18SHARUU-RUU             → Will Casper Ruud win the Shapovalov vs Ruud match?
KXATPMATCH-25OCT18COLAUG-COL             → Will Raphael Collignon win the Collignon vs Auger-Aliassime match?
KXATPMATCH-25OCT18COLAUG-AUG             → Will Felix Auger-Aliassime win the Collignon vs Auger-Aliassime match?
KXMVENFLMULTIGAMEEXTENDED-S2025870FBBF2E7A-B2B9566CF48 → yes Cole Kmet,yes Justin Herbert: 200+,yes Trevor Lawrence: 200+,yes Matthew Stafford: 200+,yes Patrick Mahomes: 175+,yes Drake Maye: 200+,yes Caleb Williams: 200+,yes Jaxson Dart: 175+,yes Jared Goff: 175+,yes Baker Mayfield: 200+,yes Cole Kmet: 10+,yes DJ Moore: 20+,yes Olamide Zaccheaus: 10+,yes Cam Skattebo: 10+,yes D'Andre Swift: 40+,yes Cam Skattebo: 40+,yes David Montgomery: 30+
KXMVENFLMULTIGAMEEXTENDED-S2025D8F8015AE6D-A4292CC515E → yes Green Bay wins by over 1.5 points,no Los Angeles R wins

In [63]:
def poll_orderbook(ticker, api_key, interval=10, limit=5):
    snapshots = []

    for i in range(limit):
        ob = fetch_current_orderbook(ticker, api_key)
        timestamp = datetime.utcnow().isoformat() + "Z"
        
        if ob:
            ob["timestamp"] = timestamp
            snapshots.append(ob)
            print(f"[{i+1}/{limit}] Snapshot at {timestamp} recorded.")
        else:
            print(f"[{i+1}/{limit}] ❌ Failed to fetch snapshot.")

        time.sleep(interval)

    return snapshots

In [64]:
snapshots = poll_orderbook(MARKET_TICKER, API_KEY, interval=15, limit=5)

#Flatten and display
df = pd.json_normalize(snapshots)
pd.set_option("display.max_columns", None)
df.head()

/var/folders/dv/l04j1r_d2mgbb4ngbgfg__0r0000gn/T/ipykernel_27715/2293755634.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().isoformat() + "Z"


[1/5] Snapshot at 2025-10-18T06:19:18.289734Z recorded.
[2/5] Snapshot at 2025-10-18T06:19:33.374872Z recorded.
[3/5] Snapshot at 2025-10-18T06:19:48.449223Z recorded.
[4/5] Snapshot at 2025-10-18T06:20:03.525772Z recorded.
[5/5] Snapshot at 2025-10-18T06:20:18.593956Z recorded.


,timestamp,orderbook.no,orderbook.no_dollars,orderbook.yes,orderbook.yes_dollars
0,2025-10-18T06:19:18.289734Z,None,None,None,None
1,2025-10-18T06:19:33.374872Z,None,None,None,None
2,2025-10-18T06:19:48.449223Z,None,None,None,None
3,2025-10-18T06:20:03.525772Z,None,None,None,None
4,2025-10-18T06:20:18.593956Z,None,None,None,None


In [62]:
df.to_csv("orderbook_snapshots.csv", index=False)